### 📥 Requisitos de Entrada

El script requiere que el usuario cargue uno o varios archivos **PDF** con listados o reportes de CPE, generalmente a través de la función interactiva de carga de archivos de Google Colab (`files.upload()`).

* **Contenido esperado del PDF:** Listados con las siguientes piezas de información en líneas:
    * **Número de CPE** (patrón: `EBxx - xxxx`).
    * **Receptor** (nombre o RUC).
    * **Importe Total** (formato monetario, ej. `S/150.00`).
    * **Fecha de Emisión** (formato: `dd/mm/yyyy`).

***

### 📤 Salida Generada

El script genera un único archivo unificado que contiene la información estandarizada de todos los PDFs procesados.

* **Nombres de archivos de salida:** Utilizan el nombre base del **primer PDF** procesado:
    1.  **CSV:** `/content/{nombre_archivo_base}_listado.csv`
    2.  **XLSX:** `/content/{nombre_archivo_base}_listado.xlsx`
* **Estructura de la tabla de salida (Orden de Columnas):**

| Columna | Descripción | Formato |
| :--- | :--- | :--- |
| **`__archivo__`** | Nombre del PDF original de donde proviene el registro. | `string` |
| **`Nro_CPE`** | Número del comprobante. | `string` |
| **`Receptor`** | Nombre o RUC del receptor. | `string` |
| **`Importe_Total`** | Monto total del comprobante. | `float` (numérico) |
| **`Fecha_Emision`** | Fecha de emisión. | `DD/MM/YYYY` (string) |

***

In [ ]:
# Instalar dependencias (solo 1ra vez)
!apt-get -qq install -y openjdk-11-jre-headless > /dev/null
!pip -q install tabula-py pandas openpyxl python-dateutil > /dev/null
!pip -q install pdfplumber pandas openpyxl python-dateutil

import io, re, os
import pandas as pd
import tabula # -- Extracción de tablas con Tabula --
from dateutil import parser
import pdfplumber
from dateutil import parser
import unicodedata, re

In [11]:
# Colab: sólo se importa si realmente vamos a subir
try:
    from google.colab import files
except Exception:
    files = None

FORCE_TEXT_PARSE = True # fuerza a ignorar tablas y usar regex por texto
REUSE_PREVIOUS_UPLOAD = True # <-- False si quieres forzar nueva subida

# patrón robusto: (EB01 - 497) ... (NOMBRE O RUC) ... (S/150.00) ... (31/01/2025)
# Se hace más flexible para capturar el texto entre Nro_CPE e Importe_Total como 'Receptor'
_line_re = re.compile(
    r"^(EB\d{2}\s*-\s*\d+)\s*" # Grupo 1: Nro_CPE (ej: EB01 - 648)
    r"(.*?)" # Grupo 2: Receptor (captura todo hasta el importe)
    r"(S\/\s*\d{1,3}(?:[.,]\d{3})*[.,]\d{2})" # Grupo 3: Importe_Total (ej: S/465.00)
    r".*?" # Cualquier cosa entre importe y fecha
    r"(\d{2}\/\d{2}\/\d{4})" # Grupo 4: Fecha_Emision (ej: 31/10/2025)
)

# --- Funciones de Extracción y Limpieza ---

def try_extract_tables(pdf):
    """Intenta extraer tablas de todas las páginas de un PDF usando pdfplumber.

    Utiliza una estrategia de extracción basada en texto para mejorar
    la robustez sin depender de las líneas de la tabla.

    :param pdf: Objeto PDF abierto con pdfplumber.
    :type pdf: :class:`pdfplumber.pdf.PDF`
    :raises: No lanza excepciones, retorna lista vacía si falla la extracción.
    :return: Lista de DataFrames de pandas, uno por cada tabla encontrada
             que tenga al menos 2 filas (cabecera + 1 dato).
    :rtype: list[pandas.DataFrame]
    """
    dfs = []
    for page in pdf.pages:
        # estrategia basada en texto (no necesita líneas de tabla)
        tables = page.extract_tables({
            "vertical_strategy": "text",
            "horizontal_strategy": "text",
            "intersection_tolerance": 5,
            "snap_tolerance": 3,
        }) or []
        for t in tables:
            if not t or len(t) < 2:
                continue
            df = pd.DataFrame(t[1:], columns=t[0])
            dfs.append(df)
    return dfs

# ---

def fallback_parse_text(pdf):
    """
    Extrae datos de CPE (Comprobantes de Pago Electrónico) del texto del PDF
    usando expresiones regulares, como mecanismo de respaldo (fallback).

    Busca líneas que contengan el patrón de número de CPE, el importe, la fecha
    y extrae el texto intermedio como el Receptor.

    :param pdf: Objeto PDF abierto con pdfplumber.
    :type pdf: :class:`pdfplumber.pdf.PDF`
    :return: DataFrame con las columnas 'Nro_CPE', 'Receptor', 'Importe_Total' y 'Fecha_Emision',
             o un DataFrame vacío si no se encuentran coincidencias.
    :rtype: :class:`pandas.DataFrame`
    """
    rows = []
    for page in pdf.pages:
        text = page.extract_text() or ""
        for raw in text.split("\n"):
            line = raw.strip()
            # salta cabeceras y líneas vacías
            if not line or line.lower().startswith(("nro.", "receptor", "importe", "fecha", "comprobante")):
                continue
            m = _line_re.search(line)
            if m:
                # Se ajusta para capturar 4 grupos
                nro, receptor_raw, imp, fecha = m.groups()
                # Limpia el receptor, removiendo espacios y guiones iniciales/finales
                receptor = receptor_raw.strip().replace(" - ", " ").strip("- ").strip()
                rows.append([nro.strip(), receptor, imp.replace(" ", ""), fecha])
    if not rows:
        return pd.DataFrame()
    # Se ajustan las columnas para incluir 'Receptor'
    return pd.DataFrame(rows, columns=["Nro_CPE", "Receptor", "Importe_Total", "Fecha_Emision"])

# ---

def _norm(s):
    """
    Normaliza una cadena de texto para comparación: quita acentos,
    convierte a minúsculas y elimina caracteres no alfanuméricos/espacios.

    :param s: Cadena de texto a normalizar.
    :type s: str
    :return: Cadena de texto normalizada.
    :rtype: str
    """
    # normaliza: sin acentos, sin espacios/puntos, minúsculas
    s = unicodedata.normalize('NFKD', str(s))
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return re.sub(r"[\W_]+", "", s).lower()

# ---

def _smart_rename(df):
    """
    Renombra automáticamente las columnas de un DataFrame a los nombres estándar
    ('Nro_CPE', 'Receptor', 'Importe_Total', 'Fecha_Emision') basándose en una
    detección de palabras clave en los encabezados normalizados.

    :param df: DataFrame de pandas con los datos extraídos.
    :type df: :class:`pandas.DataFrame`
    :return: DataFrame con las columnas renombradas.
    :rtype: :class:`pandas.DataFrame`
    """
    # mapea encabezados "variantes" -> estándar
    rename = {}
    for c in df.columns:
        k = _norm(c)
        if "nro" in k and "cpe" in k:
            rename[c] = "Nro_CPE"
        elif "receptor" in k or "ruc" in k or "cliente" in k: # Agregado 'receptor'
            rename[c] = "Receptor"
        elif "importe" in k and "total" in k:
            rename[c] = "Importe_Total"
        elif "fecha" in k and ("emision" in k):
            rename[c] = "Fecha_Emision"
    return df.rename(columns=rename)

# ---

def _as_series(df, col):
    """
    Devuelve una Serie de pandas, incluso si la selección por nombre
    de columna resulta en un DataFrame (ej. por columnas duplicadas).
    En caso de duplicados, elige la subcolumna con más valores no nulos.

    :param df: DataFrame de pandas.
    :type df: :class:`pandas.DataFrame`
    :param col: Nombre de la columna a seleccionar.
    :type col: str or list
    :return: Serie de pandas.
    :rtype: :class:`pandas.Series`
    """
    obj = df[col]
    if isinstance(obj, pd.DataFrame):
        # elige la subcolumna con más valores no nulos
        sub = obj.loc[:, obj.notna().sum().sort_values(ascending=False).index]
        return sub.iloc[:, 0]
    return obj

# ---

def clean_df(df, source_name):
    """
    Limpia, normaliza y estandariza un DataFrame extraído, asegurando
    la presencia y el formato correcto de las columnas clave:
    'Nro_CPE', 'Receptor', 'Importe_Total', 'Fecha_Emision'.

    :param df: DataFrame de pandas con los datos extraídos.
    :type df: :class:`pandas.DataFrame`
    :param source_name: Nombre del archivo PDF de origen.
    :type source_name: str
    :return: DataFrame limpio y estandarizado con las columnas
             ['__archivo__', 'Nro_CPE', 'Receptor', 'Importe_Total', 'Fecha_Emision'].
    :rtype: :class:`pandas.DataFrame`
    """
    df = _smart_rename(df).copy()
    df["__archivo__"] = source_name

    # Nro_CPE: si no existe, toma la primera columna
    if "Nro_CPE" not in df.columns:
        df["Nro_CPE"] = _as_series(df, df.columns[0]).astype(str)

    # Receptor: si no existe, intenta tomar la segunda columna
    if "Receptor" not in df.columns:
        # Intenta tomar la segunda columna si hay más de una, sino la deja vacía por ahora
        if df.shape[1] > 1:
            cand = df.columns[1]
            df["Receptor"] = _as_series(df, cand).astype(str).str.strip().replace(" - ", "-")
        else:
            df["Receptor"] = "" # Crea columna vacía si no hay más opciones

    # Importe_Total: (Lógica sin cambios)
    if "Importe_Total" not in df.columns:
        cand = None
        for c in df.columns:
            s = _as_series(df, c).astype(str)
            if s.str.contains(r"(S\/\s*)?\d{1,3}(?:[.,]\d{3})*[.,]\d{2}$", regex=True, na=False).mean() > 0.20:
                cand = c; break
        if cand is None:
            cand = df.columns[min(3, df.shape[1]-1)] # Se ajusta el índice a la 4ta columna si el receptor está
        df["Importe_Total"] = _as_series(df, cand)

    # Limpieza y conversión del importe (Lógica sin cambios)
    s = (
        _as_series(df, "Importe_Total").astype(str)
        .str.replace(r"[^\d.,-]", "", regex=True)
        .str.replace(",", "", regex=False)
        .str.replace(r"^\.$", "", regex=True)
        .str.strip()
    )
    df["Importe_Total"] = pd.to_numeric(s, errors="coerce")

    # Fecha_Emision: (Lógica sin cambios)
    if "Fecha_Emision" not in df.columns:
        cand = None
        for c in df.columns:
            s = _as_series(df, c).astype(str)
            if s.str.contains(r"\b\d{2}/\d{2}/\d{4}\b", regex=True, na=False).mean() > 0.20:
                cand = c; break
        if cand is None:
            cand = df.columns[-1]
        df["Fecha_Emision"] = _as_series(df, cand)

    # Normaliza nombres y fecha (Lógica sin cambios)
    df = df.rename(columns=lambda x: str(x).strip().replace(" ", "_"))
    df["Fecha_Emision"] = pd.to_datetime(df["Fecha_Emision"], dayfirst=True, errors="coerce")

    # Devuelve sólo las columnas estándar en el ORDEN SOLICITADO
    cols = ["__archivo__", "Nro_CPE", "Receptor", "Importe_Total", "Fecha_Emision"]
    return df[[c for c in cols if c in df.columns]]


# -------- Orquestación de entrada y Salida (Sin cambios mayores, solo se incluye para un script completo) --------

uploaded = {}
if REUSE_PREVIOUS_UPLOAD and "up" in globals() and isinstance(up, dict) and len(up) > 0:
    uploaded = up
elif files is not None:
    print("Por favor, sube uno o varios archivos PDF.")
    up = files.upload()
    uploaded = up
else:
    raise RuntimeError("No hay archivos. Debes subir archivos PDF usando files.upload() o adaptar el script para rutas locales si no estás en Colab.")

if not uploaded:
    raise RuntimeError("No se subieron archivos. Por favor, sube los archivos PDF.")

all_dfs = []

for fname in uploaded.keys():
    path = os.path.join("/content", fname) if not os.path.exists(fname) else fname

    if not os.path.exists(path):
        print(f"⚠️ No se encontró el archivo: {fname}. Saltando.")
        continue

    try:
        with pdfplumber.open(path) as pdf:
            if FORCE_TEXT_PARSE:
                tmp = fallback_parse_text(pdf)
            else:
                dfs = try_extract_tables(pdf)
                tmp = pd.concat(dfs, ignore_index=True) if dfs else fallback_parse_text(pdf)
    except Exception as e:
        print(f"❌ Error al procesar {fname} con pdfplumber: {e}")
        continue


    if tmp.empty:
        print(f"⚠️ {fname}: no se detectaron tablas ni coincidencias de líneas.")
        continue

    tmp = tmp.loc[:, ~tmp.columns.duplicated()].copy()
    tmp = clean_df(tmp, fname)
    all_dfs.append(tmp)

# -------- Salida --------
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df = final_df.drop_duplicates(subset=["__archivo__","Nro_CPE","Fecha_Emision"], keep="first")

    # Obtiene el nombre del primer archivo (sin la extensión .pdf)
    base_fname = list(uploaded.keys())[0].replace(".pdf", "")

    # Usa el nombre base para generar las rutas de salida
    csv_path = f"/content/{base_fname}_listado.csv"
    xlsx_path = f"/content/{base_fname}_listado.xlsx"

    final_df["Fecha_Emision"] = final_df["Fecha_Emision"].dt.strftime("%d/%m/%Y").fillna("")

    final_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    final_df.to_excel(xlsx_path, index=False)
    print("✅ Listo. Archivos generados:", csv_path, "y", xlsx_path)

    try:
        from IPython.display import display
        display(final_df.head(10))
    except ImportError:
        print("Muestra de las primeras 10 filas:")
        print(final_df.head(10).to_markdown(index=False))

else:
    print("⚠️ No se extrajo información utilizable de los PDFs.")

✅ Listo. Archivos generados: /content/09-SETIEMBRE___ Listado de CPE ____listado.csv y /content/09-SETIEMBRE___ Listado de CPE ____listado.xlsx


,__archivo__,Nro_CPE,Receptor,Importe_Total,Fecha_Emision
0,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 627,,365.0,30/09/2025
1,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 626,,600.0,30/09/2025
2,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 625,,560.0,30/09/2025
3,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 624,,225.0,30/09/2025
4,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 623,,165.0,30/09/2025
5,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 622,,300.0,30/09/2025
6,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 621,,405.0,30/09/2025
7,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 620,,465.0,30/09/2025
8,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 619,,600.0,30/09/2025
9,09-SETIEMBRE___ Listado de CPE ___.pdf,EB01 - 618,,480.0,29/09/2025
